In [2]:
import pycosat
import secrets
import math
import numpy as np
# import pandas
import pandas as pd
# import matplotlib
import matplotlib.pyplot as plt
# import seaborn
import seaborn as sns
%matplotlib inline

In [6]:
file = open("Scrum.dimacs")

In [7]:
lines = file.readlines()

In [4]:
names = []
for line in lines[:124]:
    names.append(line.split(' ')[2][:-1])

In [5]:
dimacs = lines[125:]

In [6]:
len(dimacs)

255

In [7]:
cnf = [[int(s)  for s in line.split(' ') if int(s) != 0] for line in dimacs]

In [8]:
solutions = []
i = 0
for sol in pycosat.itersolve(cnf):
    if i == 10000:
        break
    i += 1
    solutions.append(sol)
print(i)

10000


In [9]:
class Item:
    def __init__(self, item):
        self.r = -1
        self.d = -1
        self.theta = -1
        self.item = item
        self.features = sum(item)
        self.totalcost = secrets.randbelow(100)
        self.knowndefects = secrets.randbelow(100)
        self.featuresused = secrets.randbelow(self.features)

In [10]:
binary_solutions = [[1 if val > 0 else 0 for val in sol] for sol in solutions]

items = [Item(item) for item in binary_solutions]
max_features = -math.inf
min_features = math.inf
max_totalcost = -math.inf
min_totalcost = math.inf
max_known = -math.inf
min_known = math.inf
max_featuresused = -math.inf
min_featuresused = math.inf

for x in items:
    if x.features > max_features:
        max_features = x.features
    if x.features < min_features:
        min_features = x.features
    if x.totalcost > max_totalcost:
        max_totalcost = x.totalcost
    if x.totalcost < min_totalcost:
        min_totalcost = x.totalcost
    if x.knowndefects > max_known:
        max_known = x.knowndefects
    if x.knowndefects < min_known:
        min_known = x.knowndefects
    if x.featuresused > max_featuresused:
        max_featuresused = x.featuresused
    if x.featuresused < min_featuresused:
        min_featuresused = x.featuresused

In [11]:
def split_bin(items, total_group):
    west = []
    east = []
    westItems = []
    eastItems = []
    rand = secrets.choice(items)
    max_r = -math.inf
    min_r = math.inf
    for x in items:
        x.r = sum(x.item)
        x.d = sum([a_i - b_i for a_i, b_i in zip(x.item, rand.item)])
        if x.r > max_r:
            max_r = x.r
        if x.r < min_r:
            min_r = x.r
    for x in items:
        x.r = (x.r - min_r)/(max_r - min_r + 10**(-32))
    R = set([r.r for r in items])
    for k in R:
        g = [item for item in items if item.r == k]
        g.sort(key=lambda x: x.d, reverse=True)
        for i in range(len(g)):
            g[i].theta = (2*math.pi*(i+1))/len(g)
    thk = max_r / total_group
    for a in range(total_group):
        g = [i for i in items if (a*thk)<= i.r <= ((a+1)*thk)]
        #print(a)
        g.sort(key=lambda x: x.theta)
        if len(g) > 0:
            east.append(g[0])
            west.append(g[len(g)-1])
            for i in g:
                if i.theta <= math.pi:
                    eastItems.append(i)
                else:
                    westItems.append(i)
    return west, east, westItems, eastItems

In [12]:
from itertools import count

class TreeNode:
    _ids = count(0)

    def __init__(self, east, west, east_node, west_node):
        self.id = next(self._ids)
        self.east = east
        self.west = west
        self.east_node = east_node
        self.west_node = west_node
        self.score = 0


In [13]:
def sway(items, enough):
    if len(items) < enough:
        return TreeNode(items, None, None, None)
    delta1, delta2 = [], []
    west, east, west_items, east_items = split_bin(items, 10)
    east_node = sway(east_items, enough)
    west_node = sway(west_items,enough)
    node = TreeNode(east, west, east_node, west_node)
    return node

In [14]:
tree = sway(items, 100)

In [15]:
def difference(west, east):
    w = np.array(west.item)
    e = np.array(east.item)
    res = np.logical_xor(w, e)
    return np.sum(res)

In [16]:
def diff_array(west, east):
    w = np.array(west.item)
    e = np.array(east.item)
    res = np.logical_xor(w, e)
    return res

In [17]:
def level_rank_features(root): 
    if (not root): 
        return
    items_rank = np.zeros(len(root.west[0].item))
    # queue to hold tree node with level 
    q = []  
  
    # let root node be at level 1 
    q.append([root, 1]) 
  
    p = [] 
  
    # Do level Order Traversal of tree 
    while (len(q)):  
        p = q[0] 
        q.pop(0)
        if(p[0].west != None and p[0].east != None):
            diff = diff_array(p[0].west[0], p[0].east[0])
            for i, d in enumerate(diff):
                if d == True and items_rank[i] == 0.0:
                    items_rank[i] = p[1]
        if (p[0].west_node): 
            q.append([p[0].west_node, p[1] + 1]) 
        if (p[0].east_node): 
            q.append([p[0].east_node, p[1] + 1 ]) 
    return items_rank

In [39]:
def rank_nodes(root, rank):
    if (not root): 
        return
    largest = -100000000
    # queue to hold tree node with level 
    q = []  
    # let root node be at level 1 
    q.append([root, 1]) 
    p = [] 
    # Do level Order Traversal of tree 
    while (len(q)):  
        p = q[0] 
        q.pop(0)
        if(p[0].west != None and p[0].east != None):
            diff = diff_array(p[0].west[0], p[0].east[0])
            res = np.multiply(diff, rank)
            p[0].score = np.sum(res)
            if p[0].score > largest:
                largest = p[0].score
        if (p[0].west_node): 
            q.append([p[0].west_node, p[1] + 1]) 
        if (p[0].east_node): 
            q.append([p[0].east_node, p[1] + 1 ]) 
    return largest

In [40]:
rank = level_rank_features(tree)

In [41]:
largest = rank_nodes(tree, rank)

In [52]:
largest

22.0

In [43]:
tree.score

11.0

In [33]:
def bfs(tree, largest):
    # maintain a queue of paths
    queue = []
    # push the first path into the queue
    queue.append([tree])
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        path_id = [x.id for x in path]
        # get the last node from the path
        #import pdb; pdb.set_trace()
        node = path[-1]
        if(node.west):
            #print(diff)
            # path found
            if largest == node.score:
                return path_id, node
            # enumerate all adjacent nodes, construct a new path and push it into the queue
            neighbors = []
            if(node.west_node):
                neighbors.append(node.west_node)
            if(node.east_node):
                neighbors.append(node.east_node)
            for adjacent in neighbors:
                new_path = list(path)
                new_path.append(adjacent)
                queue.append(new_path)


In [58]:
bfs(tree, largest)

([254, 253, 189, 157], <__main__.TreeNode at 0x197e5cb5dd8>)

In [56]:
path, node = bfs(tree, largest)

In [57]:
node.score

22.0

In [38]:
rank

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 2., 0., 1., 0., 1.,
       0., 2., 3., 0., 2., 0., 3., 3., 0., 2., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 0.])

In [60]:
diff_array(node.west[0], node.east[0])

14

In [23]:
def prLevel(root): 
    tree_lvl = []
    if (not root): 
        return
  
    # queue to hold tree node with level 
    q = []  
  
    # let root node be at level 1 
    q.append([root, 1]) 
  
    p = [] 
  
    # Do level Order Traversal of tree 
    while (len(q)):  
        p = q[0] 
        q.pop(0)
        if(p[0].west != None and p[0].east != None):
            tree_lvl.append((p[1], difference(p[0].west[0], p[0].east[0]))) 
        if (p[0].west_node): 
            q.append([p[0].west_node, p[1] + 1]) 
        if (p[0].east_node): 
            q.append([p[0].east_node, p[1] + 1 ]) 
    return tree_lvl

In [ ]:
tree_data = prLevel(tree)

In [ ]:
x, cl = [], []
for cls, xs in tree_data:
    x.append(xs)
    cl.append(cls-1)
    

In [ ]:
d = {'Depth':cl,'Difference':x}
df = pd.DataFrame(d)
df.head()

In [ ]:
df = df.where(df['Depth']!= 0)

In [ ]:
ax = sns.boxplot(x="Depth", y="Difference", data=df)

In [ ]:
from collections import defaultdict
d = defaultdict(list)
for cls, x in tree_data:
    d[cls].append(x)
    

In [ ]:
d

In [ ]:
def rankFeatures(items):
    count = np.zeros(len(items[0].item))
    for item in items:
        count = np.add(count, item.item)
    rank = np.zeros(len(count))
    for i, v in enumerate(count):
        if v == 0:
            rank[i] = -1
            print("No", names[i])
        if v == (len(items)):
            rank[i] = -1
            print("All", names[i])
    return count, rank

In [ ]:
len(items)

In [ ]:
count, rank = rankFeatures(items)

In [ ]:
for i, r in enumerate(rank):
    if r == 0:
        print(names[i])
print(len(rank))

In [18]:
df = pd.read_csv('terms_sentence_map.csv')

In [23]:
df['sentence'].tolist()

[' Effort estimate based on the experience of the professionals',
 ' 5 minute sprint review meeting',
 ' Presence on scrum meetings required',
 ' 8 hours sprint review day',
 ' Only the Product Owner does prioritization',
 ' 17 day sprint',
 ' Option for virtual presence on daily meetings',
 ' Progress by product backlog id',
 ' Physical presence on scrum meetings required',
 ' Effort estimate based on the Development Team that takes the role of Scrum Master',
 ' Daily meeting every day',
 ' 21 day sprint',
 ' How to classify the status of an item in the Sprint Backlog',
 ' Effort estimate based on story points and points of value',
 ' 6 to 8 developers in team size',
 ' Option for virtual presence on daily meetings',
 ' Activity 6 of the Product owner',
 ' Activity 5 of the Product owner',
 ' The Roles used in this scrum adaptation',
 ' Activity 4 of the Product owner',
 ' Activity 3 of the Product owner',
 ' Activity 2 of the Product owner',
 ' Two Product owners for the project',
 '

In [27]:
from collections import defaultdict
from random import randint

#23 Groups
#10 students per group
#130 students
#total keys = 2990
dict = defaultdict(list)
alphabet = list(map(chr, range(97, 123)))
for letter in alphabet:
    for i in range(130):
        key = letter + str(randint(100000, 999999))
        dict[letter].append(key)
        
    

In [29]:
df = pd.DataFrame.from_dict(dict)

In [30]:
df.head()

,a,b,c,d,e,f,g,h,i,j,...,q,r,s,t,u,v,w,x,y,z
0,a235725,b742437,c115381,d467686,e356730,f376064,g627193,h448248,i988185,j416803,...,q253931,r643061,s671615,t774628,u620048,v297808,w819480,x542871,y869958,z593907
1,a534877,b998601,c872097,d488025,e685121,f711377,g431664,h187314,i769116,j910319,...,q232079,r517877,s917945,t216146,u975662,v922036,w562975,x541257,y286662,z102884
2,a626139,b965405,c812943,d930169,e649444,f848289,g169733,h252477,i967804,j299249,...,q548144,r250012,s867022,t331341,u274613,v515537,w613892,x460569,y288358,z934205
3,a880553,b184775,c889609,d473103,e284893,f679385,g892618,h103518,i408572,j983111,...,q826125,r598633,s262625,t500251,u658836,v464221,w765641,x263684,y380445,z770980
4,a485550,b453115,c822361,d107100,e353515,f258534,g723276,h583520,i794302,j180020,...,q348346,r406365,s800467,t778796,u564272,v927906,w151573,x918383,y869397,z129303


In [32]:
df.to_csv('keys.csv',index=False)